# Shallow water with bathymetry

In this notebook we study the Riemann problem for the shallow water equations in the presence of varying bathymetry.  Specifically, we consider the case in which the elevation of the bottom is piecewise constant with a jump at $x=0$.  Thus the Riemann problem consists of the shallow water equations, including an additional term in the momentum equation (corresponding to the bathymetry):
\begin{align}
    h_t + (hu)_x & = 0 \\
    (hu)_t + \left(hu^2 + \frac{1}{2}gh^2\right)_x & = -ghb_x,
\end{align}
together with initial data
\begin{align}
    (h(x,0),u(x,0)) & = \begin{cases} (h_l, u_l) & x<0 \\ (h_r, u_r) & x>0  \end{cases}
\end{align}
and bathymetry
$$b(x) = \begin{cases} b_l & x<0 \\ b_r & x>0 \end{cases}.$$

## Stationary wave at $x=0$

The source term owing to the bathymetry is (like that introduced to represent traffic flow from an on-ramp) singular: since the bathymetry $b(x)$ is discontinuous at $x=0$, the term $b_x$ is given by a delta-function:  
$$b_x = (b_r-b_l)\delta(x) = \Delta b \delta(x).$$  
Based on what we have learned in earlier chapters, it is natural to expect that the Riemann solution will include a stationary jump at $x=0$, due to this singular source term.  What are the appropriate jump conditions for this wave?  As usual, we would like to impose continuity of the flux at $x=0$, which for the first equation gives  
$$h_- u_- = h_+ u_+.$$  
where e.g. $h_\pm$ denote the limiting values of $h$ as $x\to0$ from the right and left.
The other appropriate condition is evidently
\begin{align} \label{h_ave}
    h_+ u_+^2 - h_- u_-^2 + \frac{1}{2}g(h_+^2-h_-^2) = -g\Delta b \tilde{h},
\end{align}
but it is unclear what value should be used for $\tilde{h}$.
The source term involves the product of a delta function ($b_x$) and a discontinuous function ($h$).  The product of these two functions has no clear definition.  What is the second jump condition that should be imposed at $x=0$?

The answer is still a matter of debate, and is unclear from both a mathematical perspective (as just mentioned) and from a physical perspective, since the shallow water assumption that a column of fluid has a single uniform velocity must break down near a bathymetry jump.

### Hydrostatic approach

Here we will take a natural condition that has been proposed in the literature.  That is that the momentum flux on either side of $x=0$ should be equal, after taking into account the fact that the bathymetry on the higher side supports part of the pressure of the water on the lower side.  For concreteness, suppose that $b_r > b_l$.  Then we impose
$$h_- u_-^2 + \frac{1}{2}g(h_- - (b_r-b_l))^2 = h_+ u_+^2 + \frac{1}{2}gh_+^2.$$
This expression can also be written
$$h_+ u_+^2 - h_- u_-^2 + \frac{1}{2}g(h_+^2-h_-^2) = -g(h_- - \Delta b/2)\Delta b ,$$
which has the form of \eqref{h_ave} above with $\tilde{h} = h_- - \Delta b/2$.

### Approach of Alcrudo et. al.

$$h_+ - h_- + \frac{1}{2g}(u_+^2 - u_-^2) + \Delta b = 0$$
This can also be written
$$ \frac{\tilde{h}}{2}(u_+^2 - u_-^2) + \frac{g}{2} (h_+^2 - h_-^2) = - g \tilde{h}\Delta b$$
where $\tilde{h}$ is the arithmetic average $(h_++h_-)/2$.

### Smooth steady state approach (George)

D. George (citation) showed that any two steady states in a smooth steady state solution satisfy  
$$h_+ u_+^2 - h_- u_-^2 + \frac{1}{2}g(h_+^2-h_-^2) = -g\Delta b \tilde{h},$$  
where
$$\tilde{h} = \frac{h_+ + h_-}{2} \frac{\widetilde{\lambda^+ \lambda^-}}{\overline{\lambda^+ \lambda^-}}$$
with
$$\overline{\lambda^+ \lambda^-} = \left(\frac{u_+ + u_-}{2}\right)^2 - g \frac{h_+ + h_-}{2}$$
and
$$\widetilde{\lambda^+ \lambda^-} = \max(0,u_+ u_-) - g \frac{h_+ + h_-}{2}.$$
This is equivalent to the approach of Alcrudo above (I think).

## Resonance

We expect that the Riemann solution will consist of two genuinely nonlinear waves (as in the case of the shallow water equations with flat bathymetry) and the stationary wave just described.  However, as we have seen in the chapter on traffic with a variable speed limit, these waves can interact in more interesting ways.  Note that we can write the SW with bathymetry system as a system of three equations:
\begin{align}
    h_t + (hu)_x & = 0 \\
    (hu)_t + \left(hu^2 + \frac{1}{2}gh^2\right)_x & = \frac{1}{2}ghb_x \\
    b_t & = 0.
\end{align}
Here we have added the bathymetry as a third conserved quantity.  The above system can be written as a homogeneous quasilinear hyperbolic system $q_t + A(q)q_x = 0$, with $q=[h,hu,b]^T$.  The eigenvalues of $A(q)$ are then
$$\lambda = u-\sqrt{gh}, 0, u+\sqrt{gh}.$$
This is exactly what we'd expect: the non-zero eigenvalues are the characteristic speeds present in the homogeneous shallow water system, while the zero eigenvalue corresponds to the stationary wave.  However, notice that the ordering of the eigenvalues is not fixed, since the values $u\pm\sqrt{gh}$ can be positive or negative depending on the nature of the flow.  In particular, these values can pass through zero, in which case the quasilinear system is not strictly hyperbolic -- it has a repeated eigenvalue.  This leads to the phenomenon of resonance, in which a transcritical rarefaction wave is split by the stationary wave into two waves.  One of them is a rarefaction, while the other may be a rarefaction or a shock (just as we saw in the case of the traffic flow model with varying speed limit).

Thus in general the Riemann solution may consist of as many as four waves.

## Examples

In [ ]:
%matplotlib inline
from exact_solvers import shallow_water_bathymetry as swb
import numpy as np
import matplotlib.pyplot as plt
from utils import riemann_tools

In [ ]:
def c1(q, xi, g=1.):
    "Characteristic speed for shallow water 1-waves."
    h = q[0]
    if h > 0:
        u = q[1]/q[0]
        return u - np.sqrt(g*h)
    else:
        return 0

def c2(q, xi, g=1.):
    "Characteristic speed for shallow water 2-waves."
    h = q[0]
    if h > 0:
        u = q[1]/q[0]
        return u + np.sqrt(g*h)
    else:
        return 0

### Subcritical cases

In [ ]:
b_l = 0.
b_r = 1.
h_l = 2.-b_l
h_r = 1.5-b_r
u_l = 1.
u_r = -1.
g=1.

q_l = np.array([h_l,h_l*u_l])
q_r = np.array([h_r,h_r*u_r])
states, speeds, reval, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='hydrostatic')
states, speeds, reval_d, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='alcrudo')
fig, axes = plt.subplots(1,2,figsize=(14,4))
riemann_tools.plot_waves(states,speeds,reval,wave_types,t_pointer=False,ax=axes[0])
xi = np.linspace(-5,5,500)
axes[1].plot(xi,reval(xi)[0]+(xi>0),xi,reval_d(xi)[0]+(xi>0));
plt.legend(['Hydrostatic','Alcrudo']);

In [ ]:
swb.phase_plane_plot(q_l, q_r, b_l, b_r, g=g)

In [ ]:
b_l = 0.
b_r = 1.
h_l = 1.3-b_l
h_r = 1.2-b_r
u_l = 0.
u_r = 0.2
g=1.

q_l = np.array([h_l,h_l*u_l])
q_r = np.array([h_r,h_r*u_r])
states, speeds, reval, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='hydrostatic')
states, speeds, reval_d, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='alcrudo')
fig, axes = plt.subplots(1,2,figsize=(14,4))
riemann_tools.plot_waves(states,speeds,reval,wave_types,t_pointer=False,ax=axes[0])
xi = np.linspace(-5,5,500)
axes[1].plot(xi,reval(xi)[0]+(xi>0),xi,reval_d(xi)[0]+(xi>0));
plt.legend(['Hydrostatic','Alcrudo']);

In [ ]:
swb.phase_plane_plot(q_l, q_r, b_l, b_r, g=g)

### Resonant cases

In [ ]:
b_l = 0.
b_r = 1.
h_l = 2.-b_l
h_r = 2.-b_r
u_l = 0.
u_r = 2.
g = 1.

q_l = np.array([h_l,h_l*u_l])
q_r = np.array([h_r,h_r*u_r])
states, speeds, reval, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='hydrostatic')
states, speeds, reval_d, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='alcrudo')
fig, axes = plt.subplots(1,2,figsize=(14,4))
riemann_tools.plot_waves(states,speeds,reval,wave_types,t_pointer=False,ax=axes[0])
xi = np.linspace(-5,5,500)
axes[1].plot(xi,reval(xi)[0]+(xi>0),xi,reval_d(xi)[0]+(xi>0));
plt.legend(['Hydrostatic','Alcrudo']);

In [ ]:
swb.phase_plane_plot(q_l, q_r, b_l, b_r, g=g)

In [ ]:
b_l = 0.
b_r = 1.
h_l = 2.-b_l
h_r = 1.5-b_r
u_l = 1.4
u_r = 2.0
g=1.

q_l = np.array([h_l,h_l*u_l])
q_r = np.array([h_r,h_r*u_r])
states, speeds, reval, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='hydrostatic')
states, speeds, reval_d, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='alcrudo')
fig, axes = plt.subplots(1,2,figsize=(14,4))
riemann_tools.plot_waves(states,speeds,reval,wave_types,t_pointer=False,ax=axes[0])
xi = np.linspace(-5,5,500)
axes[1].plot(xi,reval(xi)[0]+(xi>0),xi,reval_d(xi)[0]+(xi>0));
plt.legend(['Hydrostatic','Alcrudo']);

### Supercritical cases

In [ ]:
b_l = 0.
b_r = 1.
h_l = 2.-b_l
h_r = 2.-b_r
u_l = 3.
u_r = 0.
g=1.

q_l = np.array([h_l,h_l*u_l])
q_r = np.array([h_r,h_r*u_r])
states, speeds, reval, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='hydrostatic')
states, speeds, reval_d, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='alcrudo')
fig, axes = plt.subplots(1,2,figsize=(14,4))
riemann_tools.plot_waves(states,speeds,reval,wave_types,t_pointer=False,ax=axes[0])
xi = np.linspace(-5,5,500)
axes[1].plot(xi,reval(xi)[0]+(xi>0),xi,reval_d(xi)[0]+(xi>0));
plt.legend(['Hydrostatic','Alcrudo']);

In [ ]:
swb.phase_plane_plot(q_l, q_r, b_l, b_r, g=g)

In [ ]:
b_l = 0.
b_r = 1.
h_l = 2.-b_l
h_r = 5.-b_r
u_l = 3.
u_r = 2.
g=1.

q_l = np.array([h_l,h_l*u_l])
q_r = np.array([h_r,h_r*u_r])
states, speeds, reval, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='hydrostatic')
states, speeds, reval_d, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='alcrudo')
fig, axes = plt.subplots(1,2,figsize=(14,4))
riemann_tools.plot_waves(states,speeds,reval,wave_types,t_pointer=False,ax=axes[0])
xi = np.linspace(-5,5,500)
axes[1].plot(xi,reval(xi)[0]+(xi>0),xi,reval_d(xi)[0]+(xi>0));
plt.legend(['Hydrostatic','Alcrudo']);

In [ ]:
b_l = 0.
b_r = 1.
h_l = 2.-b_l
h_r = 2.-b_r
u_l = 2.
u_r = 4.
g = 1.

states, speeds, wave_types = swb.predictor(h_l, h_r, u_l, u_r, b_l, b_r, g=1.)
print(speeds)

## A problematic case

First solution; this seems to violate the energy condition (see Rosetti) when the hydrostatic interface condition is used.

In [ ]:
b_l = 0.
b_r = 1.
h_l = 2.-b_l
h_r = 1.5-b_r
u_l = 3.
u_r = 3.
g=1.

q_l = np.array([h_l,h_l*u_l])
q_r = np.array([h_r,h_r*u_r])
states, speeds, reval, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='hydrostatic')
states, speeds, reval_d, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, which='alcrudo')
fig, axes = plt.subplots(1,3,figsize=(16,4))
riemann_tools.plot_waves(states,speeds,reval,wave_types,t_pointer=False,ax=axes[0])
xi = np.linspace(-5,5,500)
axes[1].plot(xi,reval(xi)[0]+(xi>0),xi,reval_d(xi)[0]+(xi>0)); axes[1].set_title('Depth')
axes[2].plot(xi,reval(xi)[1]/reval(xi)[0],xi,reval_d(xi)[1]/reval_d(xi)[0]); axes[2].set_title('Velocity')
plt.legend(['Hydrostatic','Alcrudo']);

In [ ]:
h, hu = reval([-0.01])
h = h[0]; hu=hu[0]
u = hu/h
c1 = u-np.sqrt(g*h)
print('State just to left of x=0')
print('lambda 1: ',c1)
print('E :', 0.5*u**2+h+b_l)
print('u :', u)

In [ ]:
h, hu = reval([0.01])
h = h[0]; hu=hu[0]
u = hu/h
c1 = u-np.sqrt(g*h)
print('State just to right of x=0')
print('lambda 1: ',c1)
print('E :', 0.5*u**2+h+b_l)
print('u :', u)

Second solution: this seems to satisfy the energy condition of Rosetti.

In [ ]:
u_l = 3.
u_r = 3.
q_l = np.array([h_l,h_l*u_l])
q_r = np.array([h_r,h_r*u_r])

states, speeds, reval, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, 
                                                               which='hydrostatic',perturb_guess=True)
states, speeds, reval_d, wave_types = swb.exact_riemann_solution(q_l, q_r, b_l, b_r, g=g, 
                                                                 which='alcrudo',perturb_guess=True)

fig, axes = plt.subplots(1,3,figsize=(16,4))
riemann_tools.plot_waves(states,speeds,reval,wave_types,t_pointer=False,ax=axes[0])
xi = np.linspace(-5,5,500)
axes[1].plot(xi,reval(xi)[0]+(xi>0),xi,reval_d(xi)[0]+(xi>0)); axes[1].set_title('Depth')
axes[2].plot(xi,reval(xi)[1]/reval(xi)[0],xi,reval_d(xi)[1]/reval_d(xi)[0]); axes[2].set_title('Velocity')
plt.legend(['Hydrostatic','Alcrudo']);

In [ ]:
h, hu = reval([-0.01])
h = h[0]; hu=hu[0]
u = hu/h
c1 = u-np.sqrt(g*h)
print('State just to left of x=0')
print('lambda 1: ',c1)
print('E :', 0.5*u**2+h+b_l)
print('u :', u)

In [ ]:
h, hu = reval([0.01])
h = h[0]; hu=hu[0]
u = hu/h
c1 = u-np.sqrt(g*h)
print('State just to right of x=0')
print('lambda 1: ',c1)
print('E :', 0.5*u**2+h+b_l)
print('u :', u)

Note that both solutions seem fine from the Alcrudo/George perspective (as far as I understand).

In [ ]:
h, hu = reval([3.01])
h = h[0]; hu=hu[0]
u = hu/h
c2 = u+np.sqrt(g*h)
print('lambda 2: ',c2)
print('E :', 0.5*u**2+h+b_l)
print('u :', u)

In [ ]:
h, hu = reval([5.01])
h = h[0]; hu=hu[0]
u = hu/h
c2 = u+np.sqrt(g*h)
print('lambda 2: ',c2)
print('E :', 0.5*u**2+h+b_l)
print('u :', u)

In [ ]:
print(wave_types)
print(speeds)

In [ ]:
print(states)